# Samples datation: an hypothetical example

## The online exposure age calculator

Online tools exist to compute exposure ages based on cosmogenic radionuclide concentrations measured in surface samples, such as the "online exposure age calculator" formerly known as the "CRONUS-Earth" ([Balco et al., 2008](https://www.sciencedirect.com/science/article/pii/S1871101407000647)). It is available here: [https://hess.ess.washington.edu/](https://hess.ess.washington.edu/). 

## Surface exposure dating from samples in an upland area

Two positions P1 and P2, formerly covered by a glacier (in red on the Figure), are sampled in an upland area for CRN depth profiles. Position P2 is higher in altitude, and was presumably exhumed after P1, as result of the glacier progressive retreat. 

![samples-location](../guidelines/imgs/samples-exposure-age.png)

The <sup>10</sup>Be concentrations are given in function of depth in the table below.  

| Depth (m)          | P-1 conc. (x 10³ at/g)  | P-2 conc. (x 10³ at/g)  |
| ------------------ | ----------------------- | ----------------------- |
| 0.0                | 123                     | 328                     |
| 0.5                | 57                      | 146                     |
| 0.75               | 39                      | 98                      |
| 1                  | 28                      | 66                      |
| 1.3                | 19                      | 42                      |
| 1.6                | 13                      | 27                      |
| 2.5                | 7                       | 9                       |
| 4                  | 4                       | 4                       |
| 7                  | 3                       | 2                       |

Open the [calculator](https://hess.ess.washington.edu/) and choose "Calculate exposure ages", as we are here interested to compute surface exposure ages based on <sup>10</sup>Be concentrations. However, note that you can also compute erosion rates or calibrate production rates, based on other nuclides. 

The calculator reads text lines, defining input values, that you write or paste in the "Sample data entry" box. Once filled, simply click on the `Calculate now` button to compute the exposure age. The simplest [structure](https://hess.ess.washington.edu/math/docs/v3/v3_input_explained.html) is as follows: 

- A first line for the sample, depicting the location, the thickness and the density.
- Following lines contain nuclide concentrations, uncertainties and standardizations.

In our example, the samples can be characterized as follows: 

```
sample-1	57.3351	-6.2895	91	std	4.5	2.70	1	0.0001	2022;
sample-1	Be-10	quartz	123000	2000	KNSTD;
sample-2	57.4386	-6.3025	91	std	4.5	2.70	1	0.0001	2022;
sample-2	Be-10	quartz	328000	4000	KNSTD;
```

Copy these lines in the calculator box and compute the exposure age for each position. 

**QUESTIONS**

1. Does the result seem logical to you? Why? 
2. What could have influenced the CRN value observed at the surface ?

## Depth profiles

First, run the cell below to set the working environment (no need to understand these lines of code). If Google Colab warns about the trustfullness of the code, click on `Run anyway`.

In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display
from IPython import get_ipython
import os, sys, platform, shutil

# Define root directory according to current environment
if 'google.colab' in str(get_ipython()):

    dir_root = "/content/cosmo-training"

    if os.path.exists(dir_root):
      shutil.rmtree(dir_root)

    !git clone https://github.com/franz825/cosmo-training

else:
    # Path to root directory (repository)
    dir_root = os.path.dirname(sys.path[0])

# Import custom functions for CRN computation
sys.path.append(dir_root)
from src.functions.get_parameters_values import get_parameters_values
from src.functions.compute_depth_profile import compute_nash_index, be_accumulator

Let's convert the depth profile table into a dataframe in order to plot the data and use it for further computation.

In [2]:
# Depth profiles data, with depth in cm and concentration in at/g
depths = [0, 50, 75, 100, 130, 160, 250, 400, 700]

# Concentrations profile 1
profile_1 = pd.DataFrame({
    "depth": depths,
    "concentration": [123000, 57000, 39000, 28000, 19000, 13000, 7000, 4000, 3000],
    "profile": pd.Series(np.repeat(1, len(depths)))
    })
profile_1["data"] = "observed"

# Concentrations profile 2
profile_2 = pd.DataFrame({
    "depth": depths,
    "concentration": [328000, 146000, 98000, 66000, 42000, 27000, 9000, 1000, 2000],
    "profile": pd.Series(np.repeat(2, len(depths)))
    })
profile_2["data"] = "observed"

# Merge profiles into a single table
depth_profiles = pd.concat([profile_1, profile_2], axis = 0)

display(depth_profiles)

,depth,concentration,profile,data
0,0,123000,1,observed
1,50,57000,1,observed
2,75,39000,1,observed
3,100,28000,1,observed
4,130,19000,1,observed
5,160,13000,1,observed
6,250,7000,1,observed
7,400,4000,1,observed
8,700,3000,1,observed
0,0,328000,2,observed


Now, we can plot the concentrations observed in each depth profile. 

**QUESTIONS**

1. What profile seems the oldest ? 

2. Is it logical with respect to the process of glacier retreat ?  

In [3]:
depth_profiles["profile_string"] = depth_profiles["profile"].astype(str)

# Plot depth profiles
plot = px.scatter(depth_profiles, x = "concentration", y = "depth", color = "profile_string", labels = dict(concentration = "<sup>10</sup>Be concentration (at/g)", depth = "Depth (cm)", profile_string = "Profile"))
plot.update_yaxes(autorange = "reversed")
plot.update_layout(xaxis = {'side': 'top'})
plot.show()

The aim is now to find the pair of values for erosion value and exposure age that best fit the two depth profiles, using the Nash-Suthcliffe optimization. To achieve this goal, we will model the depth profile for different combinations of erosion and exposure age values, as defined below: 

In [4]:
# Erosion values (cm)
erosion_values = [0, 100, 200, 500, 1000, 2000]
print("Erosion (cm): ")
display(erosion_values)

# Exposure age values (a)
exposure_ages = [20000, 40000, 60000, 80000, 100000, 120000, 140000]
print("Exposure ages (a): ")
display(exposure_ages)

# Parameters of CRN modelling
parameters = get_parameters_values()
print("Parameters for CRN modelling: ")
display(parameters)

Erosion (cm): 


[0, 100, 200, 500, 1000, 2000]

Exposure ages (a): 


[20000, 40000, 60000, 80000, 100000, 120000, 140000]

Parameters for CRN modelling: 


{'atn': 160,
 'prod_rate': 4.25,
 'rho': 2.7,
 'half_life_Be10': 1387000.0,
 'L': 4.997456240519106e-07}

Let's loop within the defined values to run the depth profile modelling, and collect modelled concentrations along with Nash-Sutcliffe values. The optimization is achieved for the two profiles independently.

### Profile 1

Below, we optimize the profile 1.

In [5]:
# CRN concentrations for profile 1
concentrations_profile_1 = depth_profiles.loc[depth_profiles["profile"] == 1]["concentration"].to_numpy()

# Container for optimization results
fit_profile_1 = pd.DataFrame()

# Loop within erosion values 
for i in range(len(erosion_values)):

    # Loop within exposure ages
    for j in range(len(exposure_ages)):

        # Run optimization for current set of values
        crn_fit = be_accumulator(concentrations_profile_1, depths, exposure_ages[j], erosion_values[i], 0, parameters)
        # Collect result into temporary dataframe
        current_fit = pd.DataFrame({"erosion": erosion_values[i], "exposure_age": exposure_ages[j], "nash_index": [crn_fit["nash_index"]]})
        # Collect current result in to main container
        fit_profile_1 = pd.concat([fit_profile_1, current_fit], axis = 0)

display(fit_profile_1)

,erosion,exposure_age,nash_index
0,0,20000,0.800996
0,0,40000,0.787255
0,0,60000,-0.805103
0,0,80000,-3.929132
0,0,100000,-8.538973
0,0,120000,-14.589828
0,0,140000,-22.037939
0,100,20000,0.195369
0,100,40000,0.795772
0,100,60000,0.998191


Select the pair of values that provides a Nash-Sutcliffe value closest to 1, i.e. the largest value, that will correspond to our best scenario, optimizing the exposure age and the erosion of the observed depth profile.

In [6]:
# Get row for which nash_index has the maximum value in the column. 
fit_profile_1[fit_profile_1.nash_index == fit_profile_1.nash_index.max()]

,erosion,exposure_age,nash_index
0,200,100000,0.999677


Let's now select the best scenario and get the modelled <sup>10</sup>Be concentrations at each depth.

In [7]:
# Erosion of selected scenario
erosion = 200
# Exposure age of selected scenario
exposure_age = 100000

# Run the scenario with the selected values
crn_fit_1 = be_accumulator(concentrations_profile_1, depths, exposure_age, erosion, 0, parameters)

# Get the fitted CRN values and add them to profile table
display(pd.concat([profile_1, pd.Series(crn_fit_1["crn_fitted"], name = "crn_fitted", dtype = "Float64")], axis = 1))

,depth,concentration,profile,data,crn_fitted
0,0,123000,1,observed,124734.513385
1,50,57000,1,observed,57159.152735
2,75,39000,1,observed,39501.56293
3,100,28000,1,observed,27850.08304
4,130,19000,1,observed,18942.883721
5,160,13000,1,observed,13483.809577
6,250,7000,1,observed,6636.502885
7,400,4000,1,observed,4122.139062
8,700,3000,1,observed,2689.173003


And we can plot the modelled CRN values along with the observed ones.

In [8]:
# Re-arrange the data to provide a table suitable for a proper plot
profile_1_fit = pd.DataFrame({"depth": depths, "concentration": pd.Series(crn_fit_1["crn_fitted"], name = "crn_fitted", dtype = "Float64"), "profile": pd.Series(np.repeat(1, len(depths)))})
profile_1_fit["data"] = "fitted"
# Append data to observed values
profile_1_plot = pd.concat([profile_1, profile_1_fit], axis = 0)

# Plot depth profile
plot = px.scatter(profile_1_plot, x = "concentration", y = "depth", color = "data", labels = dict(concentration = "<sup>10</sup>Be concentration (at/g)", depth = "Depth (cm)", data = "Data"), width = 800, height = 600)
plot.update_yaxes(autorange = "reversed")
plot.update_layout(xaxis = {'side': 'top'}, title = "Profile 1")
plot.show()

## Profile 2

Now, we will apply the same workflow for the second profile. 

In [9]:
# CRN concentrations for profile 2
concentrations_profile_2 = depth_profiles.loc[depth_profiles["profile"] == 2]["concentration"].to_numpy()

# Container for optimization results
fit_profile_2 = pd.DataFrame()

# Loop within erosion 
for i in range(len(erosion_values)):

    # Loop within exposure ages
    for j in range(len(exposure_ages)):

        # Run optimization for current set of values
        crn_fit = be_accumulator(concentrations_profile_2, depths, exposure_ages[j], erosion_values[i], 0, parameters)
        # Collect result into temporary dataframe
        current_fit = pd.DataFrame({"erosion": erosion_values[i], "exposure_age": exposure_ages[j], "nash_index": [crn_fit["nash_index"]]})
        # Collect current result in to main container
        fit_profile_2 = pd.concat([fit_profile_2, current_fit], axis = 0)

display(fit_profile_2)

,erosion,exposure_age,nash_index
0,0,20000,0.087491
0,0,40000,0.605330
0,0,60000,0.907250
0,0,80000,0.999568
0,0,100000,0.888458
0,0,120000,0.579952
0,0,140000,0.079940
0,100,20000,-0.257647
0,100,40000,0.081189
0,100,60000,0.364980


Select the pair of values that provides a Nash-Sutcliffe value closest to 1, i.e. the largest value, that will correspond to our best scenario, optimizing the observed depth profile.

In [10]:
# Get row for which nash_index has the maximum value in the column. 
fit_profile_2[fit_profile_2.nash_index == fit_profile_2.nash_index.max()]

,erosion,exposure_age,nash_index
0,0,80000,0.999568


Let's now select the best scenario and get the modelled <sup>10</sup>Be concentrations at each depth.

In [11]:
# Erosion of selected scenario
erosion = 0
# Exposure age of selected scenario
exposure_age = 80000

# Run the scenario with the selected values
crn_fit_2 = be_accumulator(concentrations_profile_2, depths, exposure_age, erosion, 0, parameters)

# Get the fitted CRN values and add them to profile table
display(pd.concat([profile_2, pd.Series(crn_fit_2["crn_fitted"], name = "crn_fitted", dtype = "Float64")], axis = 1))

,depth,concentration,profile,data,crn_fitted
0,0,328000,2,observed,333293.135856
1,50,146000,2,observed,146600.2041
2,75,98000,2,observed,98008.069855
3,100,66000,2,observed,66073.318921
4,130,42000,2,observed,41818.265033
5,160,27000,2,observed,27113.499591
6,250,9000,2,observed,9331.918028
7,400,1000,2,observed,4040.902037
8,700,2000,2,observed,2447.536313


And we can plot the modelled CRN values along with the observed ones.

In [12]:
# Re-arrange the data to provide a table suitable for a proper plot
profile_2_fit = pd.DataFrame({"depth": depths, "concentration": pd.Series(crn_fit_2["crn_fitted"], name = "crn_fitted", dtype = "Float64"), "profile": pd.Series(np.repeat(2, len(depths)))})
profile_2_fit["data"] = "fitted"
# Append data to observed values
profile_2_plot = pd.concat([profile_2, profile_2_fit], axis = 0)

# Plot depth profile
plot = px.scatter(profile_2_plot, x = "concentration", y = "depth", color = "data", labels = dict(concentration = "<sup>10</sup>Be concentration (at/g)", depth = "Depth (cm)", data = "Data"), width = 800, height = 600)
plot.update_yaxes(autorange = "reversed")
plot.update_layout(xaxis = {'side': 'top'}, title = "Profile 2")
plot.show()

## Interpretation 

Based on the Nash-Sutcliffe optimization criterion (best value is a maximization and should approach as close as possible to 1), how would you describe the main steps of the landscape history?